In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count, sum, to_date

spark = (SparkSession.builder
         .appName("mySparkAnalyse")
         .getOrCreate()
         )
df = spark.read.csv("web_server_logs.csv", header=True, inferSchema=True)

# |ip|timestamp|method|url|response_code|response_size|

# Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP
top_active_ip = (df.groupBy("ip")
                 .agg(count("*").alias("request_count"))
                 .orderBy(desc(col("request_count")))
                 .limit(10)
                 .select("ip", "request_count")
                 )

print("Top active IP adresses:")
top_active_ip.show()

# Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода
http_method_count = (df.groupBy("method")
                   .agg(count("*").alias("method_count"))
                   .select("method", "method_count")
                   )

print("Request count by HTTP method:")
http_method_count.show()

# Профильтруйте и посчитайте количество запросов с кодом ответа 404
# Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате

total_error_count = df.filter(df["response_code"] == 404).count()

error_count_by_day = (df.filter(df["response_code"] == 404)
                       .withColumn("date", to_date(col("timestamp")))
                       .groupBy("date")
                       .agg(sum(col("response_size")).alias("total_response_size"))
                       .orderBy("date")
                       .select("date", "total_response_size")
                       )

print(f"Number of 404 response codes: {total_error_count}")
print("Total response size by day:")
error_count_by_day.show()

spark.stop()

Top active IP adresses:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|   60.52.23.203|            2|
|   191.4.249.24|            2|
| 80.131.203.222|            2|
| 139.148.99.188|            1|
|196.139.224.190|            1|
| 160.234.183.99|            1|
|   44.243.78.11|            1|
| 173.243.113.99|            1|
| 20.247.253.147|            1|
|145.219.135.117|            1|
+---------------+-------------+

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25047|
|DELETE|       25045|
|   PUT|       25112|
|   GET|       24796|
+------+------------+

Number of 404 response codes: 25108
Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2025-01-01|            2132697|
|2025-01-02|            2046096|
|2025-01-03|            2031725|
|2025-01-04|            1979701|
|2025-01-05|            1